In [1]:
import sys
import os
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "..")))
from Raydium.adapter.RaydiumAdapter import RaydiumAdapter

adapter = RaydiumAdapter("../Raydium/config/solana.raydium.yaml")
res = adapter.check_position_exists_tool(
    position_nft_mint="GBYTTW6jMYGZUN5p4sLCVR14LMtaJqLWpjzD79k7FyLw"
    )
print(res)

2025-09-14 15:54:41,631 INFO [RaydiumAdapter] RaydiumAdapter inicializado (cluster=mainnet-beta, endpoints=1)


{'exists': True, 'details': {'bump': [255], 'fee_growth_inside_0_last_x64': '7828903933548108', 'fee_growth_inside_1_last_x64': '1963056009606400', 'liquidity': '86835618986', 'nft_mint': 'GBYTTW6jMYGZUN5p4sLCVR14LMtaJqLWpjzD79k7FyLw', 'padding': [0, 0, 0, 0, 0, 0, 0], 'pool_id': '3ucNos4NbumPLZNWztqGHNFFgkHeRMBQAVemeeomsUxv', 'recent_epoch': 848, 'reward_infos': [{'growth_inside_last_x64': '22829906762133', 'reward_amount_owed': 40876}, {'growth_inside_last_x64': '0', 'reward_amount_owed': 0}, {'growth_inside_last_x64': '0', 'reward_amount_owed': 0}], 'tick_lower_index': -15351, 'tick_upper_index': -13344, 'token_fees_owed_0': 11884142, 'token_fees_owed_1': 2844684}}
